# Plot-o-matic Story Generator

## Step 1: Title + Plot + Genre + Medium + Visual Style + Location + Time Period

The first thing we will do is give the LLM a prompt and from there have some [few shot prompts](https://www.promptingguide.ai/techniques/fewshot) to give examples on what kind of response we need. Hopefully the LLM will pick up on the pattern and return what we want.

We will use this technique to extract a **title** for the story, then the plot overview. 

Once we have that, we will take a guess as to which of these, the story falls into:

- **Medium**: e.g. Movie, Video Game, Poem
- **Genre**: e.g. Mystery, Platformer, Fairy Tale
- **Visual Style**: e.g. Realistic, Cartoon, Anime
- **Time Period**: e.g. Victorian, Roaring Twenties
- **Location**: e.g. Fantasy world, United States, Viking Scandinavia, Space

## Load Story
Here is our story so far as we left off from the previous step.

In [10]:
import settings
from model import Story, Character, CharacterRelationship
from utils import merge_models, show_diff

story_step_1 = Story.load_from_directory(settings.STORY_DIR + "/step_1")

story_step_1.display()

### Story Details

```json
{
    "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
    "title": "",
    "video": false,
    "visual_style": "",
    "time_period": "",
    "location": "",
    "genre": "",
    "medium": "",
    "plot_overview": "",
    "narrative_perspective": "",
    "conflict_type": "",
    "themes": [],
    "motifs": [],
    "characters": [],
    "props": [],
    "story_beats": [],
    "subplots": [],
    "emotional_arc": [],
    "acts": []
}
```

## Powering Up the Text Models

Now let’s bring the text models to life! Here, we set up the core LLM that will handle all chat and instruction-based interactions. Check out [model_text.py](./model_text.py) to see the mechanics under the hood.

A key setting to experiment with is the `TEMPERATURE` in [settings.py](settings.py). This controls the model’s creative flair: set it to `0` for consistent, repeatable results, or dial it up to add a dash of unpredictability! While we'll start at `0` for stability, around `0.6` can make the outputs more dynamic if you’re looking to spice things up.

Another important setting, `max_new_tokens`, determines how long the generated text can be. A higher value gives you more depth but can impact performance, so adjust wisely.

In [11]:
# Pull the models if we are using Ollama
import settings
if settings.TEXT_MODEL_BACKEND == "ollama":
    !ollama pull {settings.TEXT_MODEL}

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling c147388e9931... 100% ▕████████████████▏  42 GB                         
pulling 4863fe3335f3... 100% ▕████████████████▏ 1.2 KB                         
pulling 64e1b2889b78... 100% ▕████████████████▏ 7.6 KB                         
pulling a568f2ebc73c... 100% ▕████████████████▏ 4.7 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 2b4e98e1c22e... 100% ▕████████████████▏  562 B                         
verifying sha256 digest 
writing manifest 
success 


# NeMo Guardrails: Keeping the Story on Track

NeMo Guardrails are our trusty boundaries, guiding the story generation and keeping things safe and streamlined. They ensure our content stays suitable, preventing anything NSFW from slipping in, while also maintaining the correct formatting of our `Story` objects. With NeMo Guardrails in place, we’re free to explore creativity without sacrificing structure or appropriateness—keeping the story focused, polished, and ready to shine!

In [2]:
# # from model_text import llm, llm_json, display_messages
# from llama_index.core.llms import ChatMessage
# from IPython.display import Markdown, display
# from model_text import llm
# from guardrails import story_rails_llm

# import nest_asyncio
# nest_asyncio.apply()

# llm_rails = story_rails_llm(llm)

# prompt = [
#     ChatMessage(role="system", content="You are a helpful assistant, respond to user queries with very short and terse answers."),
#     ChatMessage(role="user", content="What is the greatest invention?"),
# ]

In [3]:
# Quick deterministic test to make sure temperature 0.0 always gives the same output
# display(Markdown("#### Deterministic Test responses:\n\n"))
# display([llm.chat(prompt, max_new_tokens=300).message.content for x in range(5)])

In [4]:
# Quick deterministic test to make sure temperature 0.0 always gives the same output
# display(Markdown("#### Deterministic Test responses:\n\n"))
# display([llm.chat(prompt, max_new_tokens=300).message.content for x in range(5)])

## Diffs
To show the evolution of the story, we will use a convenience function to show the diffs between stories.

In [12]:
from utils import show_diff
from model import Story

story1 = Story(prompt="Test prompt", title="Test Title")
story2 = Story(prompt="Test prompt", title="Test Title", plot_overview="Test Plot Overview")

show_diff(story1, story2)

```diff
--- story.json
+++ new_story.json
@@ -1,4 +1,5 @@
 {
   "prompt": "Test prompt",
-  "title": "Test Title"
+  "title": "Test Title",
+  "plot_overview": "Test Plot Overview"
 }
```

## Plot Overview

Here is a our first few-shot example. We will give some example inputs and outputs and it will pick up on the pattern. We will use `include` and `exclude` to select which properties the LLM sees.

In [13]:
# Plot Overview Generation
from model import Story
from model_text import llm, display_messages
import json
from IPython.display import display, Markdown
from utils import merge_models, show_diff, deindent
from llama_index.core.llms import ChatMessage


# Function to generate the plot overview using message lists (few-shot examples)
def generate_plot_overview(story: Story) -> str:
    system_message = deindent("""
        You are a very creative writer who generates detailed and engaging plot summaries for stories based on an input prompt. 
        Your goal is to provide a comprehensive plot overview by expanding upon the 'prompt' field in the provided story JSON object. 
        Use the prompt as a starting point to craft a compelling narrative, enriching it with rich details, characters, settings, and plot developments.

        - Make the plot overview coherent, imaginative, and captivating.
        - Include strong, relatable characters with clear goals and motivations.
        - Introduce a **central conflict that challenges the protagonist and creates tension.
        - Build rising tension and suspense by introducing obstacles and increasing stakes.
        - Show character development and growth throughout the story.
        - Create an immersive setting that enhances the narrative.
        - Incorporate themes and messages that add depth to the story.
        - Include surprising plot twists to keep the reader engaged.
        - Provide a satisfying resolution that offers closure.
        - Avoid repeating the exact wording from the prompt; instead, build upon its ideas.
        - Do not include any reasoning, explanations, or questions in your response.
        - **Output only a valid JSON object with a single property 'plot_overview', which contains your plot summary as a string.**
        - You muse escape special characters like line breaks "\\\\n", quotes \\" and tabs \\t and any other characters that will break json parsing in your JSON string.
        - Only use ASCII characters.
        - Do not include any additional text outside the JSON object.
    """)
    # Few-shot examples to guide the model
    few_shot_examples = [
        Story(
            prompt="A warrior from the future must protect a mystical kingdom from a time-traveling villain",
            plot_overview=deindent("""
                In the year 3045, Earth is on the brink of destruction due to overpopulation and environmental decay. The once-great civilization has retreated to floating cities in space, seeking refuge from the dying planet. Among these people are the last remaining warriors who still honor the values and principles of their ancestors, vowing to protect their world from any threats that may arise.

                One day, an ancient prophecy reveals the impending doom of a mystical kingdom called Elarisia from an evil time-traveling villain named Lord Draxor. To save this realm and the balance of their timeline, the warriors send one of their finest, a skilled fighter known as Zoran, back in time to the year 1200 to defend the kingdom.

                Upon arrival in Elarisia, Zoran discovers that Lord Draxor has already arrived and is working alongside King Alaric to control the land's magical resources for his own nefarious purposes. As Zoran learns more about the realm and its people, he realizes that the key to defeating Lord Draxor lies in understanding the importance of unity and balance between humans, magical creatures, and nature.

                Zoran begins his mission by protecting King Alaric's daughter, Princess Lila, who possesses unique powers and is the last descendant of a lineage crucial to maintaining harmony within Elarisia. As they journey together, Zoran trains Lila in combat and teaches her about the significance of balance among all living beings.

                Together, they face various obstacles and challenges, including navigating treacherous landscapes, battling Draxor's army of supernatural minions, and uncovering hidden secrets about the origins of both their world and his own. Along the way, Zoran uncovers the truth behind Lord Draxor's origin: he is a rogue time-traveler twisted by power and greed, hellbent on dominating all realms and destroying any potential threats to his control.

                As the climax approaches, King Alaric and Zoran devise a plan to defeat Lord Draxor once and for all. With Princess Lila's powers and the support of Elarisia's magical beings, they confront Draxor in a final epic battle that will determine the fate of their world.

                Through courage, sacrifice, and teamwork, Zoran and Lila successfully defeat Lord Draxor, restoring balance to Elarisia and saving it from eternal darkness. The future warrior returns to his time, having not only saved a mystical kingdom but also rediscovered the true value of unity and harmony between all living beings.
            """)
        ),
        Story(
            prompt="A team of scientists must survive on an alien planet after a research mission goes horribly wrong.",
            plot_overview=deindent("""
                A team of elite scientists embarks on a groundbreaking research mission to explore an uncharted alien planet teeming with potential for supporting life. Led by Dr. Elena Martinez, the crew includes experts in various fields, each eager to make history. However, upon arrival, a catastrophic malfunction causes their spaceship to crash-land, leaving them stranded with limited supplies and no way to contact Earth.

                As they assess their situation, they encounter the planet's hostile environment filled with dangerous flora and fauna unlike anything they've seen before. Tensions rise within the group as they struggle to survive harsh weather conditions, dwindling resources, and internal conflicts. Days turn into weeks as the scientists discover mysterious ruins suggesting an ancient civilization once thrived there.

                Dr. Martinez becomes obsessed with uncovering the secrets of the planet, believing it holds the key to their survival. The team must navigate treacherous terrains, decode alien technology, and confront their own fears and weaknesses. In a race against time, they work together to repair their ship using alien materials and knowledge gleaned from the ruins.

                Ultimately, they must decide whether to return home or stay and continue exploring the profound mysteries of the planet that has both threatened and captivated them. Through their journey, they learn valuable lessons about cooperation, resilience, and the ethical implications of their mission.
            """)
        ),
        Story(
            prompt="Write a book report on bioluminescent creatures",
            plot_overview=deindent("""
                In the remote and mysterious island of Nautilus, an extraordinary discovery is made - a diverse ecosystem teeming with bioluminescent creatures. Dr. Alexandra Rayne, a renowned marine biologist, embarks on an expedition to study these fascinating beings that emit light as a means of survival and communication. As the island's lush rainforests, tranquil oceans, and hidden caverns are illuminated by these incredible creatures, Dr. Rayne's research uncovers the interconnectedness and mutual dependency of this glowing ecosystem.

                Dr. Rayne is in awe as she encounters various species, including bioluminescent jellyfish, sea turtles, crustaceans, and even a rare glowing dragonfly. She discovers that these creatures have adapted to their environment by using light emission for camouflage, predator evasion, and prey attraction, creating a delicate balance between predators and prey.

                However, the idyllic setting takes a dark turn when an unscrupulous corporation, NeonLux, seeks to exploit the bioluminescent creatures' properties for profit. The company aims to harness their energy for use in human technology, disregarding the environmental consequences and ethics of such an endeavor. As Dr. Rayne delves deeper into her research, she becomes increasingly committed to protecting these unique organisms and their habitat from NeonLux's nefarious plans.

                In a thrilling race against time, Dr. Rayne teams up with local environmental activists, indigenous people who have lived in harmony with the island for generations, and a rogue scientist from NeonLux seeking redemption. Together, they devise a plan to expose NeonLux's unethical practices and preserve Nautilus Island and its bioluminescent inhabitants.

                Through courage, ingenuity, and a deep respect for nature, they succeed in safeguarding the island, highlighting themes of environmental conservation, ethical responsibility, and the wonders of the natural world.
            """)
        ),
        Story(
            prompt="Video game where an axolotl makes a friend",
            title="Axolotl Adventures",
            plot_overview=deindent("""
                In this whimsical and heartwarming video game, players follow the adventure of an axolotl named Axo, a fascinating amphibian known for its unique appearance and regenerative abilities. Initially lonely, Axo longs for friendship in their vibrant underwater world.

                The story begins with Axo exploring their environment, meeting various creatures, and encountering challenges along the way. As Axo navigates through different levels and overcomes obstacles, they discover an enchanted artifact that allows them to communicate with other species. This leads to meeting Fin, a charming little fish with a big personality.

                Fin and Axo quickly become inseparable, bonding over their shared curiosity and love for adventure. Together, they traverse various underwater realms, encountering both friends and foes. The duo faces challenges like rescuing kidnapped aquatic friends, uncovering hidden treasures, and defeating fearsome adversaries in a race to save their endangered world.

                As the story unfolds, Axo and Fin's friendship becomes central to overcoming obstacles and protecting their vibrant home. Along the way, they learn the importance of loyalty, teamwork, and the power of friendship. The game's captivating narrative combines elements of puzzle-solving, role-playing, and action-adventure genres, providing a delightful experience for players who appreciate the unique bond between Axo and Fin.
            """)
        ),
    ]

    # Combine system message, few-shot examples, and user prompt
    messages = [ChatMessage(role="system", content=system_message)]

    # Tell the LLM what fields to include in the JSON output
    fields_wanted = ["plot_overview"]

    for example in few_shot_examples:
        # Make a copy of the overview and remove it from the example to prevent leakage
        messages.append(ChatMessage(role="user", content=example.model_dump_json(exclude=["prompt"])))
        messages.append(ChatMessage(role="assistant", content=example.model_dump_json(include=fields_wanted)))

    # Add the user's prompt to generate the plot overview
    messages.append(ChatMessage(role="user", content=story.model_dump_json(exclude=fields_wanted)))

    display_messages(messages)
    response = llm.chat(messages, max_new_tokens=300)
    response_json = response.message.content
    # response_json = response_json.replace('\n', '\\n').replace('\r', '\\r')

    # Pretty print the response JSON as markdown
    # display(Markdown(f"```json\n{json.dumps(json.loads(response_json), indent=4)}\n```"))

    try:
        overview = Story.model_validate_json(response_json)
        story_with_overview = merge_models(story, overview)
        show_diff(story, story_with_overview)

        return story_with_overview
    except Exception as e:
        display(Markdown(f"### Error: {e}"))
        display(Markdown(f"```\n{response_json}\n```"))
        display(Markdown(f"```\n{json.dumps(json.loads(response_json), indent=4)}\n```"))
        raise
        


In [14]:
# Quick test
import settings
if settings.EXECUTE_EXAMPLES:
    display(Markdown(f"### Quick Story Plot tests:"))

    test_stories = [
        Story(prompt="A warrior from the future must protect a mystical kingdom from a time-traveling villain"),
        Story(prompt="Comic book scientists must survive on an alien planet after a research mission goes horribly wrong"),
    ]

    for test_story in test_stories:
        plot_overview = generate_plot_overview(test_story)
        display(Markdown(f"Prompt Input:\n\n```{test_story.prompt}```\n\nPlot Overview Output:\n\n```{plot_overview}```\n\n---\n\n"))


## Generate a plot overview for our story
Tests look good. Make one for our story in progress.

In [16]:
story_with_overview = generate_plot_overview(story_step_1)
story_with_overview.save_to_directory(settings.STORY_DIR + "/step_2")

```diff
--- story.json
+++ new_story.json
@@ -1,3 +1,4 @@
 {
-  "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told."
+  "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
+  "plot_overview": "In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.\n\nAs Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.\n\nTogether, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.\n\nAs the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.\n\nWith the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered."
 }
```

In [30]:
from IPython.display import Markdown, display

display(Markdown(story_with_overview.plot_overview))

In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.

As Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.

Together, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.

As the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.

With the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered.

In [31]:
print(settings.TEXT_MODEL)
print(settings.TEXT_MODEL_BACKEND)

nemotron:70b
ollama


## Story Title
Now that we have a general plot overview, let's make a name for the story with that plot in mind.

In [19]:
positive_examples = [
    Story(
        prompt="An astronaut stranded on a planet where time moves backward.",
        plot_overview=deindent("""
            Captain Mira Alvarez, a renowned astronaut, crash-lands on a distant planet during a deep-space exploration mission. She soon discovers that on this mysterious world, time flows in reverse. The dead come back to life, and the sun rises in the west. As she navigates this perplexing environment, Mira begins to experience her own memories unraveling.

            Determined to return home before she forgets who she is, Mira encounters Kaelen, an alien who ages backward. Together, they embark on a journey to reach a temporal vortex at the planet's core, which might be Mira's only hope of escape. Along the way, they face temporal anomalies, reverse predators, and the moral dilemma of altering time.

            As Mira's past and future blur, she must decide between seizing a chance to change her own history or preserving the integrity of time. The story explores themes of identity, the nature of existence, and the irreversible flow of time.
        """),
        title="Echoes of Yesterday"
    ),
    Story(
        prompt="A librarian discovers that the books in her library are gateways to parallel universes.",
        plot_overview=deindent("""
            Emma Sterling, a quiet librarian in a small town, stumbles upon a hidden section in the library basement. She opens an ancient book and is instantly transported to a parallel universe within its pages. Realizing that each book contains a portal to a different world, Emma is exhilarated by the possibilities.

            Her adventures take her to realms of magic, futuristic cities, and lands ruled by sentient animals. However, she soon learns that a malevolent force called The Eraser seeks to destroy these worlds by burning the books. Teaming up with characters from various stories - a warrior princess, a robotic detective, and a mischievous pixie - Emma sets out to stop The Eraser.

            Together, they journey through literary landscapes to protect the realms of imagination. Emma discovers her own courage and the profound impact stories have on reality. The tale celebrates the power of literature, creativity, and the connections that bind all worlds together.
        """),
        title="The Library of Worlds"
    ),
    Story(
        prompt="A retired superhero's pets team up to save the city when their owner goes missing.",
        plot_overview=deindent("""
            In the bustling metropolis of Arcadia City, retired superhero Thunderbolt enjoys a quiet life with his loyal pets: Zeus the dog, Whiskers the cat, and Polly the parrot. One day, Thunderbolt mysteriously disappears, and chaos erupts as old villains resurface to wreak havoc.

            Realizing their beloved owner is missing, the trio discovers that they have absorbed some of Thunderbolt's powers over the years. Zeus gains super strength, Whiskers develops telekinetic abilities, and Polly can mimic any sound to perfection. Together, they form an unlikely team to protect the city.

            Navigating challenges like operating gadgets with paws and staying under the radar, the pets uncover a plot by the nefarious Dr. Chaos to erase superheroes from history. In a race against time, they must rescue Thunderbolt and save Arcadia City. The story combines humor, action, and heart, highlighting themes of friendship, courage, and the hero within us all.
        """),
        title="Paws and Claws: Pet Heroes Unite"
    ),
    Story(
        prompt="A musical where the characters communicate only through dance and instrumental music.",
        plot_overview=deindent("""
            In the vibrant city of Melodia, words have vanished, and inhabitants express themselves through dance and music. Elena, a gifted violinist, feels out of place in a world dominated by elaborate dances. She longs to bridge the gap between melodies and movement.

            When a mysterious silence threatens to engulf Melodia, muting all sound, Elena teams up with Darius, a passionate dancer. Together, they embark on a quest to find the lost Symphony Scroll, said to hold the key to restoring harmony. Their journey takes them through rhythm-filled forests, jazz-infused deserts, and hip-hop bustling towns.

            As they face challenges that test their resolve, Elena and Darius discover the power of collaboration. By combining their talents, they create a new form of expression that can save their world. The story celebrates diversity, unity, and the universal language of art.
        """),
        title="Silent Beats"
    ),
    Story(
        prompt="An inventor builds a machine that turns dreams into reality, with unexpected consequences.",
        plot_overview=deindent("""
            Brilliant but reckless inventor Marcus Kane creates the Reverie Engine, a device that materializes dreams into the physical world. At first, the invention brings wonder - artists bring masterpieces to life, and architects construct impossible buildings. Marcus gains fame and fortune, basking in his success.

            However, as people's subconscious fears and nightmares begin to manifest, chaos ensues. Shadowy creatures roam the streets, and personal anxieties threaten global safety. Realizing his creation is spiraling out of control, Marcus teams up with psychologist Dr. Lena Moore to shut down the machine.

            They delve into a shared dreamscape to confront the collective fears of humanity. In this surreal landscape, they battle manifestations of guilt, regret, and dread. Marcus must face his own inner demons to dismantle the Reverie Engine from within. The story explores the duality of human nature, responsibility, and the thin line between dreams and reality.
        """),
        title="Waking Shadows"
    ),
    Story(
        prompt="A comedy about aliens misunderstanding Earth's customs after intercepting reality TV broadcasts.",
        plot_overview=deindent("""
            When the distant planet of Xenon intercepts Earth's reality TV shows, they believe these broadcasts depict genuine human culture. Intrigued, they send a team of aliens disguised as humans to Earth to learn more. The team includes Zog, the enthusiastic leader; Vee, the skeptic; and Blip, who can mimic any sound.

            Landing in Los Angeles, they attempt to fit in by emulating outrageous reality TV behavior - staging fake dramas, participating in talent competitions, and offering unsolicited makeovers. Their antics cause confusion and hilarity among real humans, who think they're eccentric performers.

            As the aliens bungle their way through misunderstandings, they form unexpected friendships and begin to see the richness of genuine human experiences. Realizing their error, they must decide whether to return home or stay and learn what being human truly means. The story is a lighthearted satire on media, perception, and the search for authenticity.
        """),
        title="Reality Check"
    ),
    Story(
        prompt="A historical fiction about a secret guild of artisans who can infuse emotions into objects.",
        plot_overview=deindent("""
            In Renaissance Italy, young apprentice Lucia joins the Aurora Guild, a secret society of artisans capable of embedding emotions into their creations. Paintings that evoke profound sorrow, sculptures that inspire love, and garments that instill courage - these are the works of the guild.

            When a power-hungry nobleman, Count Vero, discovers their abilities, he seeks to exploit them to control the masses. As guild members begin to disappear, Lucia teams up with Marco, a roguish thief with a heart of gold, to protect their craft.

            Together, they unravel a conspiracy that threatens the very fabric of society. Lucia must master her unique gift to create a masterpiece capable of awakening the people's spirit. The story weaves art, magic, and history, highlighting themes of freedom, expression, and the impact of creativity on the human soul.
        """),
        title="The Artisans' Secret"
    ),
    Story(
        prompt="A reclusive mathematician finds patterns that predict future events, causing global chaos.",
        plot_overview=deindent("""
            Dr. Adrian Sloan, a brilliant but introverted mathematician, discovers a complex equation that can predict future events with uncanny accuracy. Initially using it to make minor life improvements, he quickly attracts the attention of governments and corporations eager to harness this power.

            As news of his discovery leaks, the world's balance begins to tilt. Stock markets crash, political tensions rise, and society teeters on the brink of collapse as factions fight to control the equation. Realizing the catastrophic consequences, Adrian retreats, destroying his work to prevent further misuse.

            Joined by Maya, a skeptical journalist who becomes an unexpected ally, Adrian embarks on a mission to restore order. They must outwit those pursuing them and convince the world that the future should remain unwritten. The story delves into themes of fate versus free will, the ethical responsibilities of knowledge, and the unpredictability of human nature.
        """),
        title="The Equation of Destiny"
    ),
    Story(
        prompt="An environmental fable about animals forming an alliance to stop humans from destroying their habitat.",
        plot_overview=deindent("""
            In the lush Whispering Woods, animals of all species live in harmony until deforestation threatens their home. Led by Luna the fox, a council of animals - including Oliver the owl, Bella the bear, and Mia the rabbit - decide to take action.

            Unable to communicate directly with humans, they devise clever plans to hinder the destruction: rerouting rivers, orchestrating "haunted" forests, and highlighting the environmental impact through unexpected means. A young girl named Sophie, daughter of the logging company's CEO, stumbles upon their efforts.

            Moved by the animals' plight, Sophie becomes their voice, advocating for conservation. Together, they inspire a movement that challenges corporate greed. The story is a heartwarming fable about unity, the importance of protecting nature, and the power of a small group's determination to make a big difference.
        """),
        title="Voices of the Forest"
    ),
    Story(
        prompt="A thriller where people's shadows start acting independently, revealing hidden truths.",
        plot_overview=deindent("""
            In the city of Umbra, a strange phenomenon occurs: people's shadows begin acting independently, exposing their innermost secrets and desires. Detective Sarah Grey, known for her logical approach, is assigned to investigate the inexplicable events causing panic and chaos.

            As shadows reveal crimes, betrayals, and hidden talents, society unravels. Sarah's own shadow starts communicating with her, hinting at a past she has long suppressed. Teaming up with Dr. Elias Warren, a fringe scientist studying the link between consciousness and the physical world, they search for answers.

            They uncover that an experimental energy project has fractured the barrier between the subconscious and reality. With time running out, Sarah must confront her own demons to restore order. The story explores themes of self-awareness, the duality of human nature, and the consequences of playing with forces beyond our understanding.
        """),
        title="Shadowfall"
    ),
]


In [20]:
from model import Story
from model_text import llm, display_messages
from utils import merge_models, show_diff, deindent
import textwrap
from unidecode import unidecode
from llama_index.core.llms import ChatMessage

def generate_story_name(story: Story) -> Story:
    system_message = deindent("""
        You are a creative writer tasked with generating unique, captivating and imaginative titles for story oververviews. 
        Your goal is to provide a concise and engaging title that reflects the essence of the story based on the provided prompt and plot_overview fields in the JSON object.

        - Use the 'prompt' and 'plot_overview' properties of the incoming object to craft a title that is original and enticing.
        - Keep the title concise - ideally under 10 words.
        - Avoid using colons ':' or dashes '-' in the title.
        - Only use ASCII characters.
        - Do not include any reasoning, explanations, or additional commentary.
        - Output only a valid JSON object with the 'title' attribute populated.
        - Do not include any additional text outside the JSON object.
                              
        **ONLY RETURN A VALID JSON OBJECT WITH title PROPERTY AND NOTHING ELSE**
    """)

    # System message
    messages = [ChatMessage(role="system", content=system_message)]

    # Use few-shot examples to guide the model
    for sample in positive_examples:
        # Include 'prompt' and 'plot_overview' in the user's message, exclude 'title'
        user_content = sample.model_dump_json(include={"prompt", "plot_overview"})
        # Assistant's response includes only the 'title'
        assistant_content = sample.model_dump_json(include={"title"})

        messages.extend([
            ChatMessage(role="user", content=user_content),
            ChatMessage(role="assistant", content=assistant_content),
        ])

    # User prompt without any existing title
    user_input = story.model_dump_json(include={"prompt", "plot_overview"})
    user_input = unidecode(user_input)
    messages.append(ChatMessage(role="user", content=user_input))

    # Display messages if needed
    # display_messages(messages)

    # Call your LLM inference function to generate the title
    response = llm.chat(messages, max_new_tokens=50)
    response_json = response.message.content

    # Parse the JSON response
    try:
        titled_story = Story.model_validate_json(response_json)
    except Exception as e:
        display(Markdown(f"---\n---\n### Error: {e}"))

        display_messages(messages)

        display(Markdown(f"### plot overview: \n{story.plot_overview}]\n\n"))
        
        display(Markdown(f"### Broken json: \n```json\n{response_json}\n```"))
        
        display(Markdown(f"```\n{response}\n```"))
        raise
    
    story_with_title = merge_models(story, titled_story)
    
    show_diff(story, story_with_title)
    
    # Successfully parsed the LLM response
    return story_with_title


In [21]:
# Quick test
import settings

if settings.EXECUTE_EXAMPLES:
    display(Markdown(f"### Quick Story Name tests:"))

    test_stories = [
        Story(
            prompt="A warrior from the future must protect a mystical kingdom from a time-traveling villain",
            plot_overview=deindent("""
                In a distant future, Earth's final warrior, Kai, discovers a secret portal that transports him to the ancient kingdom of Eldoria. Here, magic reigns supreme, and the land is under threat from a time-traveling tyrant named Lord Malakar. Sent from the future to rewrite history, Malakar's arrival has destabilized time itself, causing chaos across multiple eras.

                Kai, a skilled fighter unaccustomed to magic, must adapt quickly to protect Eldoria. He joins forces with Seraphine, the kingdom's fierce sorceress, and a band of legendary heroes. Together, they battle through enchanted forests, treacherous castles, and shifting sands of time to prevent Malakar from reshaping the past. Along the journey, Kai learns that his mission is tied to a prophecy - he is the 'Eternal Guardian' destined to preserve all realms.

                As Kai and his allies confront Malakar in a climactic battle, they face their own pasts, testing their courage and unity. In the end, Kai must choose between returning to his future or staying to safeguard Eldoria forever.
            """),
        ),
        Story(
            prompt="Comic book scientists must survive on an alien planet after a research mission goes horribly wrong",
            plot_overview=deindent("""
                Dr. Alexis Nova and her team of scientists set out to study the mineral-rich alien planet Exovent, only to have their shuttle malfunction and crash upon arrival. Stranded on a planet covered in vivid, bioluminescent jungles and strange, living metals, they soon realize Exovent is far more hostile than they anticipated. 

                With dwindling resources and no way to contact Earth, Alexis must lead her team through deadly terrains and face encounters with bizarre alien creatures. They discover ruins suggesting an ancient civilization once thrived there, but it vanished without a trace. As they delve deeper into Exovent's mysteries, the team uncovers clues about the planet's sentience and its ability to manipulate matter.

                Facing both environmental dangers and psychological strain, Alexis and her crew must decipher Exovent's secrets before it consumes them. This comic book thriller blends science fiction and survival, pitting humanity's resilience against the unknown.
            """),
        ),
        Story(
            prompt="A movie: Five stuffed animals come to life and go on a quest to find their missing owner",
            plot_overview=deindent("""
                One stormy night, in a cozy attic, five beloved stuffed animals - Buttons the Bear, Luna the Rabbit, Scout the Fox, Willow the Elephant, and Tinker the Tortoise - awaken to find their young owner, Lucy, mysteriously missing. Bound by loyalty and love, they venture into the sprawling world beyond the attic to find her.

                As they journey through bustling city streets, eerie forests, and crowded playgrounds, they face challenges that test their courage and teamwork. Along the way, they meet other lost toys and forge unlikely alliances. Guided by clues Lucy left behind, they begin to unravel a hidden message that leads them to the heart of their city, where Lucy is waiting in a hospital, dreaming of her toys.

                In a heartwarming reunion, they find Lucy, bringing comfort and joy to her through their presence. The film captures themes of love, resilience, and the magic that exists in childhood friendships.
            """),
        ),
        Story(
            prompt="a parable about a lion and a giraffe",
            plot_overview=deindent("""
                In the vast plains of the Serengeti, Leo the Lion rules with strength and pride, while Gemma the Giraffe finds peace in the tall trees. Though they live in the same kingdom, they rarely cross paths until a drought threatens their land.

                As water and food become scarce, Leo is forced to rely on Gemma, whose height gives her access to the last remaining leaves. Despite their differences, they embark on a journey to find a new oasis, learning about each other's unique strengths and weaknesses along the way.

                Through their shared struggles, Leo realizes that true leadership requires compassion, and Gemma discovers her own courage. Together, they teach the other animals about the power of cooperation, leaving a legacy of unity and understanding that reshapes their kingdom forever.
            """),
        ),
        Story(
            prompt="a video game about mummies vs zombies",
            plot_overview=deindent("""
                In an ancient Egyptian tomb, a forgotten ritual accidentally awakens a group of mummies, protectors of a sacred amulet. At the same time, a viral outbreak in the modern world causes the dead to rise as zombies, hungry and mindless.

                Players take on the roles of ancient mummies, each with unique powers derived from ancient spells and relics. The mummies must defend their tomb from an onslaught of zombies trying to steal their amulet, which contains the power to unleash darkness upon the world.

                Battling through crypts, desert ruins, and enchanted sands, the mummies unlock new powers and confront zombie bosses in epic showdowns. The game challenges players to strategize and combine magical attacks to prevent the apocalypse, blending humor, action, and undead combat in a battle for ancient honor.
            """),
        ),
        Story(
            prompt="make a greeting card that is hilarious about cats. give it a funny title. maybe a one-liner",
            plot_overview=deindent("""
                A quirky greeting card featuring a cat wearing sunglasses and lounging on a couch with the tagline: 'I was going to get you something fabulous... but then I took a nap.' Inside, it reads, 'Happy Purr-thday! Remember: naps first, plans later.'

                Perfect for cat lovers, this card combines humor with relatable cat habits, making it ideal for birthdays or just to bring a smile.
            """),
        ),
        Story(
            prompt="little pig in the big city",
            plot_overview=deindent("""
                A curious little pig named Pip leaves his quiet farm for the bustling streets of Big City. Overwhelmed but determined, Pip encounters honking cars, towering buildings, and strange city folk. Along the way, he befriends a stray dog, a mischievous pigeon, and a street musician, who teach him the ropes of urban life.

                As Pip learns the city's quirks and charms, he inadvertently becomes a hero, rescuing lost pets and uncovering a plot against the local animal shelter. By the end of his adventure, Pip returns to the farm, forever changed and braver than before, with memories of the big city and new friends.
            """),
            title="Little Pig, Big Dreams"
        ),
        Story(
            prompt="kid story in the woods", 
            plot_overview=deindent("""
                In a quiet village nestled at the edge of an enchanted forest, there lived a young girl named Mira. The villagers often told stories of the forest - how it was alive with magic, where the trees whispered secrets and creatures of legend roamed freely. Though everyone was warned never to enter, Mira felt a strange connection to it.

                One evening, just as the sun dipped behind the trees, she saw a flicker of light deep within the forest, a golden glow unlike anything she had ever seen. Without a second thought, she followed it, venturing past the tree line. As she walked deeper into the woods, the trees parted before her, revealing a hidden path.

                At the end of the path stood a towering, ancient oak with a door carved into its trunk, the light coming from within. Heart pounding, Mira stepped closer, realizing that her adventure had only just begun, and the stories the village had told weren't the whole truth - there was far more magic to be discovered.
            """),
            title="The Whispering Forest"
        ),
    ]

    for test_story in test_stories:
        title = generate_story_name(test_story)
        display(Markdown(f"# Title Output:\n\n \n>{title}\n\n---\n\n"))
        display(Markdown(f"```json\n{title.model_dump_json(include=["prompt", "plot_overview"])}\n```"))

## Set the title for our story
The tests look pretty good. Let's update our story with a title, too.

In [22]:
# Generate a story name
story_with_title = generate_story_name(story_with_overview)
story_with_title.save_to_directory(settings.STORY_DIR + "/step_2")

```diff
--- story.json
+++ new_story.json
@@ -1,4 +1,5 @@
 {
   "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
+  "title": "Beyond the Pixel Veil",
   "plot_overview": "In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.\n\nAs Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.\n\nTogether, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.\n\nAs the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.\n\nWith the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered."
 }
```

## Decide Genre + Medium + Visual Style + Time Period + Location
We do this separate from title generation to give more creative freedom to the LLM agents collaborating. The interesting part here is we have multiple fields, so we are getting JSON out.

The JSON parsing has room for improvement since the LLM can sometimes be pretty unpredictable.

In [23]:
# These are just examples and not an exhaustive list
mediums = ['Book', 'Movie', 'Show', 'Video Game', 'Board Game', 'Comic', 'Graphic Novel', 'Play', 'Musical', 'Podcast', 'Poem', 'Visual Novel', 'Short Film', 'Interactive Fiction', 
           'Board Game', 'Tabletop RPG', 'Novella', 'Song', 'Webcomic', 'Animation', 'Vlog', 'Documentary', 'Virtual Reality', 'Painting', 'Sculpture', 'Installation Art', 'Photography', 
           'Web Series', 'Audio Drama', 'Dance Performance', 'Radio Play', 'VR Experience', 'Augmented Reality Experience']
genres = ['Adventure', 'Fantasy', 'Science Fiction', 'Dystopian', 'Cyberpunk', 'Steampunk', 'Romance', 'Historical Fiction', 'Mystery', 'Thriller', 'Horror', 'Gothic', 'Paranormal', 
          'Urban Fantasy', 'High Fantasy', 'Low Fantasy', 'Epic Fantasy', 'Space Opera', 'Hard Science Fiction', 'Post-Apocalyptic', 'Speculative Fiction', 'Western', 'Noir', 
          'Slice of Life', 'Magical Realism', 'Biography', 'Memoir', 'Autobiography', 'Self-Help', 'Travel', 'True Crime', 'History', 'Philosophy', 'Science', 'Journalism', 
          'Romantic Comedy', 'Action-Adventure', 'Science Fantasy', 'Dark Fantasy', 'Superhero', 'Historical Romance', 'Horror Comedy', 'Psychological Thriller', 'Fantasy Romance', 
          'Techno-Thriller', 'Role-Playing Game', 'Action RPG', 'First-Person Shooter', 'Simulation', 'Puzzle', 'Platformer', 'Survival', 'Fighting', 'Real-Time Strategy', 
          'Turn-Based Strategy', 'Massively Multiplayer Online', 'Open-World', 'Visual Novel', 'Sports', 'Rhythm', 'Heist', 'Espionage', 'Military', 'Satire', 'Musical', 
          'Mythology', 'Political Thriller', 'Social Commentary', 'Fairy Tale', 'Folklore']
visual_styles = ['hyperrealistic photo', 'cinematic Neo-Noir style with vibrant color contrasts', 'high-detail, realistic fantasy', 'cinematic dark mystery', 
                 'hyperrealistic highly detailed, gritty urban with desaturated colors', 'retro 1980s style with vibrant neon colors and heavy grain texture', 
                 'Classic American Cartoon', 'Manga', 'Rubber Hose', 'Western Comic Book', 'Disney Cartoon', '3D CGI cartoon', 'Anime', 'Sketchy Line Art Cartoon', 'claymation', 
                 'Graphic Novel', 'Abstract Surrealist', 'Fantasy Storybook', 'Retro Futuristic Cartoon', 'Children Picture Book' , 'Chibi', 'Pixel Art', 'Cel-Shaded', '2D Hand-Drawn', 
                 '3D CGI', 'Watercolor', 'Pencil Sketch', 'Vector Art', 'Impressionist', 'Abstract', 'Pop Art', 'Minimalist', 'Steampunk', 'Dieselpunk', 'Cyberpunk', 'Gothic', 'Vintage/Retro', 
                 'Surrealism', 'Fantasy', 'Dark Fantasy', 'Noir', 'Art Deco', 'Low Poly', 'Grunge', 'Manga', 'Fantasy Realism', '8-bit', '16-bit', 'Graffiti Art', 'Collage', 'Flat Design', 
                 'Motion Graphics', 'Expressionism', 'Neon Art', 'Stencil Art', 'Watercolor or Painterly Style', 'Digital Cel Shading', 'Chalkboard Animation', 'Retro Vintage (50s/60s) cartoon', 
                 'Noir/Black & White comic book', 'Noir/Black & White movie', 'Classic Superhero comic', 'Steampunk Gothic comic', 'vintage tattoo', 'Classic Saturday Morning cartoon' , 
                 'Cutout Collage Style', 'Low-Poly 3D game', 'Cel Shaded video game', 'Top-Down Retro RPG game', 'Isometric View Fantasy game', 'board game', 'Survival Horror Realism game', 
                 'Voxel Art', 'Sci-Fi Cyberpunk', 'Western Gritty Realism', 'Sci-Fi Realism', 'realistic cinematic Retro 80s Neon', 'cinematic Fantasy Medieval', 'High Fantasy Realism', 
                 'cinematic Cyberpunk Futurism', 'Cyberpunk Futurism cartoon', 'Cyberpunk Futurism comic', 'cinematic Dark Psychological Thriller', 'Dark Psychological Thriller comic']
time_periods = ['Renaissance', 'Victorian Era', 'Roaring Twenties', 'Great Depression', 'World War II', 'Cold War', 'Future', '1800s', "Prehistoric Era", "Classical Greece", 
                "Medieval Period", "Ancient Egypt", "Dark Ages", "Age of Exploration", "World War I", "World War II", "1970s Counterculture", "1980s Cyberpunk Era", "1990s Grunge", 
                "Dystopian Future", "Stone Age", "Retro-Futurism", "prehistoric"]
locations = ["Fantasy World", "United States", "Viking Scandinavia", "Space", "Medieval Kingdom", "Post-Apocalyptic Wasteland", "Ancient Egypt", "Steampunk City", "Alien Planet", 
             "Underwater Atlantis", "Wild West Frontier", "Dystopian Future Metropolis", "Haunted Mansion", "Cyberpunk Mega-City", "Renaissance Italy", "Jungle Island", "Pirate Cove", 
             "Frozen Tundra", "Mythical Greece", "Feudal Japan", "Wild West", "Classical Greece", "Subterranean", "Deserted Island", "Futuristic Mars Colony", "Urban Jungle", "Jungle", 
             "Urban Slum District", "Ancient Mesopotamia", "Ancient Mayan Ruins", "Pirate Ship"]


In [24]:
# Positive examples - Show the LLM what to do
positive_examples = [
    Story(
        prompt="A warrior from the future must protect a mystical kingdom from a time-traveling villain",
        genre="Fantasy",
        medium="Book",
        video=False,
        visual_style="Epic Illustrated Fusion Illustration - Intricate Detail and Fine Line Work, Bold, Vibrant Color Palette with Gradients, Dynamic Framing with Sweeping, Cinematic Perspectives",
        time_period="Distant Future",
        location="Mystical Kingdom",
        plot_overview="A futuristic warrior is transported back in time to a magical kingdom, where they must battle a villain who uses time travel to destabilize the realm."
    ),
    Story(
        prompt="A team of scientists must survive on an alien planet after a research mission goes horribly wrong",
        genre="Science Fiction",
        medium="Movie",
        visual_style="Hyperrealistic Cinematic Futuristic Sci-Fi Realism photo - Cinematic Lighting with Cold, Neon Highlights and Atmospheric Effects",
        time_period="Near Future",
        video=True,
        location="Alien Planet",
        plot_overview="Stranded on a hostile alien planet, a team of scientists must use their wits and limited resources to survive and uncover the planet's secrets."
    ),
    Story(
        prompt="A detective investigates a series of strange disappearances in a haunted town",
        genre="Mystery",
        medium="TV Show",
        video=True,
        visual_style="Cinematic Dark Noir photo - Chiaroscuro Lighting with Deep Shadows, Distorted Reflections and Smoke Effects",
        time_period="Modern Day",
        location="Haunted Town",
        plot_overview="In a small, eerie town with a history of hauntings, a detective digs into mysterious disappearances that may have supernatural causes."
    ),
    Story(
        prompt="A hacker steals money from a bank in a high-tech city of the future",
        genre="Cyberpunk",
        medium="Game",
        video=True,
        visual_style="Realistic photo of Neon Cyber Noir Video Game - Dark, High-Contrast Color Palette with Neon Highlights, Heavy Use of Light and Shadow",
        time_period="Near Future",
        location="High-Tech City",
        plot_overview="In a sprawling cyberpunk metropolis, a skilled hacker takes on the corporate banking system, facing high-tech security and rival hackers in their quest."
    ),
    Story(
        prompt="A group of friends go on a road trip across the country",
        genre="Adventure",
        medium="Show",
        video=True,
        visual_style="Whimsical Americana Cartoon Illustration - Soft, Pastel or Retro Color Palette, Simplified, Rounded Shapes and Lines",
        time_period="Modern Day",
        location="United States",
        plot_overview="A close-knit group of friends embarks on a cross-country road trip, encountering quirky towns, interesting people, and personal growth along the way."
    ),
    Story(
        prompt="Write a short, simple nursery rhyme for young children about stars dancing in the night sky. The rhyme should be playful, comforting, and easy to remember, with a soft, gentle tone. It should encourage children to imagine the stars twinkling and dancing as they fall asleep. Include repeating lines and a soothing rhythm",
        genre="Nursery Rhyme",
        medium="Poem",
        video=False,
        visual_style="N/A",
        time_period="Timeless",
        location="Night Sky",
        plot_overview="A soothing, rhythmic nursery rhyme that invites children to imagine stars dancing above, filling them with a sense of wonder and calm as they drift to sleep."
    ),

    Story(
        prompt="A brave, super-powered hero defends a bustling metropolis from a wave of ruthless villains, each with unique powers and vendettas against the city. When a sinister mastermind unites these villains in a plot to destroy the city, the hero must rally allies, use clever tactics, and face impossible odds to protect their home.",
        genre="Superhero Action",
        medium="Comic Book",
        video=False,
        visual_style="Classic American Superhero Comic illustration - bold, inky outlines, vibrant, saturated colors",
        time_period="Near Future",
        location="Neon Metropolis",
        plot_overview="The Silver Sentinel, a hero with the power to manipulate light, faces her greatest challenge yet as a coordinated attack on the city threatens innocent lives and the very structure of her home. Villains like the shadowy Nightcrawler, the icy Frostbite, and the electricity-wielding Volt wreak havoc across the metropolis. With every street and skyscraper in danger, the Silver Sentinel gathers a small team of heroes and sets out to take down the villains one by one. In an epic final showdown at the city's tallest building, the Sentinel faces the mastermind orchestrating the chaos - a former ally turned foe with a grudge."
    ),
    Story(
        prompt="In a dystopian city plagued by corruption, a cynical private investigator takes on a mysterious case involving a missing scientist. The deeper he digs, the more he uncovers a conspiracy that reaches into the highest levels of power, ultimately putting his life at risk",
        genre="Neo-Noir Mystery Thriller",
        medium="Graphic Novel",
        video=False,
        visual_style="Cinematic Neo-Noir Graphic Novel Illustration - Stylized Line Work and Cross-Hatching",
        time_period="Dystopian Near Future",
        location="The Underbelly of Neo-City",
        plot_overview="Detective Elias Moore, a down-on-his-luck investigator, is hired by the family of a missing scientist rumored to have been working on a revolutionary project. Navigating the dark, sprawling streets of Neo-City, Elias uncovers a trail of deceit, betrayal, and violence as he encounters corrupt politicians, rogue corporate enforcers, and strange experiments gone wrong. Each clue reveals that the scientist may have been on the brink of uncovering a secret that powerful figures are willing to kill to protect. As Elias wrestles with his own past and deteriorating moral code, he closes in on the truth, leading to a tense showdown in the fog-covered industrial district."
    ),
]

In [25]:
# Function to generate a genre for the story prompt using few-shot examples
from model_text import llm_json
from utils import merge_models, show_diff

# Errors is a list of exceptions that occurred during the generation process, so the model can adjust
def generate_story_genre(story: Story) -> Story:
    system_message = (
        "You are a story classifer that generates concise, cohesive, creative and relevant attributes based on the attributes of a story. "
        "Keep your responses short and terse. Only include one json object in your response and nothing else. "
        "You will classify the genre, medium, location, time_period and visual style. Create one for each category that best fits the story. "
        "If the story has moving images, like a video game or movie, animated should be set to true."
        "If the story is live action, the visual style should be hyperrealistic.\n "
        "The following lists are just a place to start and you are not limited to that. You have full creative freedom as long as it doesn't contradict the story so far. "
        f"Here are some examples of genres: {', '.join(genres)}\n"
        f"Here are some examples of mediums: {', '.join(mediums)}\n"
        f"Here are some examples of time periods: {', '.join(time_periods)}\n"
        f"Here are some examples of locations: {', '.join(locations)}\n"
        f"Here are some examples of visual styles: {', '.join(visual_styles)}\n\n"
        "The visual_style should give queues like color pallete, whether it is an illuatration or photo, lighting, and other visual elements. "
        "Do not include any explanations, commentary, or reasoning in your response. \n"
        "**Respond with ONLY the JSON object and NOTHING ELSE.**"
    )

    messages = [ChatMessage(role="system", content=system_message)]
    fields_wanted = ["genre", "medium", "visual_style", "location", "time_period"]

    # Show some example input and some example outputs
    for example in positive_examples:
        # User prompt
        messages.append(ChatMessage(role="user", content=example.model_dump_json(exclude=fields_wanted)))
        # LLM response
        messages.append(ChatMessage(role="assistant", content=example.model_dump_json(include=fields_wanted)))

    # Now the user input
    messages.append(ChatMessage(role="user", content=story.model_dump_json(exclude=fields_wanted)))
    
    # Call your LLM inference function to generate the genre
    classification = llm_json.chat(messages, max_new_tokens=500).message.content

    # Parse the JSON response
    classified_story = Story.model_validate_json(classification)
    story_with_classification = merge_models(story, classified_story)
    show_diff(story, story_with_classification)
    
    # Successfully parsed the LLM response
    return story_with_classification

In [26]:
# Quick test
import settings
if settings.EXECUTE_EXAMPLES:
    test_genre_prompts = [
        Story(prompt="A young girl must navigate a world where robots have taken over, and humanity is in hiding."),
        Story(prompt="A group of childhood friends go on an adventure to uncover hidden treasure."),
        Story(prompt="a video game where an axolotl makes a friend", title="Axolotl Adventures", plot_overview="Players take on the role of an adventurous young axolotl named Ollie, who sets out on an epic journey to find a true friend in the vibrant, mysterious underwater world of Lake Xochimilco. As Ollie explores various environments - from dense kelp forests to ancient underwater ruins - he encounters a diverse array of creatures, each with unique abilities and personalities. Players must solve puzzles, complete challenges, and help other animals along the way, building relationships that eventually lead Ollie to his new best friend. With charming characters, heartwarming moments, and exciting quests, the game celebrates the power of friendship and teamwork in overcoming obstacles."),
        Story(prompt="a painting of dogs playing cards"),
        
    ]
    for test_genre_prompt in test_genre_prompts:
        genre = generate_story_genre(test_genre_prompt)
        display(Markdown(f"Prompt Input:\n\n```\n{test_genre_prompt.short_context()}\n```\n\nGenre Output:\n\n```\n{genre.classification_context()}\n```\n\n---\n\n"))

## Update our story
Update our story with a genre and such, too.

`generate_story_genre` returns a new story with the populated fields, so we will call `merge_models` to copy those to the original story object.

In [27]:
# Let's get the genre and such for our global story
story_with_classification = generate_story_genre(story_with_title)

```diff
--- story.json
+++ new_story.json
@@ -1,5 +1,10 @@
 {
   "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
   "title": "Beyond the Pixel Veil",
+  "visual_style": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Dynamic Panel Layouts, Intricate Digital Patterns, and Soft Glow Effects",
+  "time_period": "Near Future/Dystopian Alternate Reality",
+  "location": "New Mumbai (Earth) / Nexus (Alternate Reality)",
+  "genre": "Science Fantasy/Young Adult Gaming Thriller",
+  "medium": "Novel with Illustrated Chapters",
   "plot_overview": "In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.\n\nAs Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.\n\nTogether, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.\n\nAs the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.\n\nWith the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered."
 }
```

# Save the Story
Let's keep our progress so far.

In [28]:
story_with_classification.save_to_directory(settings.STORY_DIR + "/step_2")
story_with_classification.display()

### Story Details

```json
{
    "prompt": "In a world where virtual reality gaming competitions have become the new norm, Meera, a teenage underdog gamer, discovers her life is intertwined with an alternate reality of cosmic proportions. \nAs she learns to harness her extraordinary skills, Meera forms an unlikely team with fellow gamers she was at odds with previously.\nFacing the harsh truth that her actions in this alternate universe directly impact events on Earth, Meera confronts the internal struggle of knowing she has taken lives. \nEmbracing her destiny as a hero, Meera and her diverse team navigate through challenges, deception, and danger to uncover the truth behind the lies they've been told.",
    "title": "Beyond the Pixel Veil",
    "video": false,
    "visual_style": "Vibrant, Neon-Lit Cyberpunk Manga Illustration - Dynamic Panel Layouts, Intricate Digital Patterns, and Soft Glow Effects",
    "time_period": "Near Future/Dystopian Alternate Reality",
    "location": "New Mumbai (Earth) / Nexus (Alternate Reality)",
    "genre": "Science Fantasy/Young Adult Gaming Thriller",
    "medium": "Novel with Illustrated Chapters",
    "plot_overview": "In the immersive world of Eon, where virtual reality gaming competitions have captivated the globe, 17-year-old Meera 'Midnight' Singh is an unassuming underdog gamer from the outskirts of New Mumbai. Her life takes a drastic turn when she discovers that her exceptional gaming skills are, in fact, a conduit to an alternate reality known as the Nexus - a realm where celestial entities clash, and the fabric of space-time hangs in the balance.\n\nAs Meera delves deeper into the mysteries of the Nexus, she's forced to put aside past rivalries and form an uneasy alliance with her gaming nemeses: Jax 'Specter' Lee, a charismatic streaming sensation; Dr. Zhang 'Zen' Wei, a soft-spoken yet brilliant gamer with a hidden agenda; and Maya 'Rampart' Patel, a fierce and enigmatic player with unparalleled defensive skills.\n\nTogether, they must navigate the blurred lines between their virtual reality gaming world, Eon, and the Nexus. Meera soon confronts the devastating truth: every decision she makes in the Nexus has catastrophic consequences on Earth, including the loss of human lives. Torn between her passion for gaming and the weight of her newfound responsibility, Meera must come to terms with the moral implications of her actions.\n\nAs the team progresses through the treacherous landscape of the Nexus, they encounter formidable foes, ancient deities, and hidden forces manipulating the gamers from the shadows. Meera's quest for answers leads her to the Architects, enigmatic creators of Eon and the Nexus, who harbor secrets that threaten to upend everything she thought she knew about reality.\n\nWith the fate of two worlds hanging in the balance, Meera and her diverse team must put aside their differences, trust each other, and master their unique skills to prevent a cosmic catastrophe. Will Meera embrace her destiny as a hero, or will the pressures of her dual existence tear her apart? The journey through Eon and the Nexus is just the beginning, where the boundaries between gaming, reality, and legend are forever altered.",
    "narrative_perspective": "",
    "conflict_type": "",
    "themes": [],
    "motifs": [],
    "characters": [],
    "props": [],
    "story_beats": [],
    "subplots": [],
    "emotional_arc": [],
    "acts": []
}
```

# Next Step
Onto [Step 3: Character Descriptions](./3_character_descriptions.ipynb)